In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import itertools as it
%matplotlib inline

In [ ]:
flow = 10

In [ ]:
G = nx.MultiDiGraph()
travel_time_awal = [10,20,25]
key = G.add_edge(1,2, capacity = 2)
key2 = G.add_edge(1,2, capacity = 4)
key3 = G.add_edge(1,2, capacity = 3)


In [ ]:
df = nx.to_pandas_edgelist(G)
df['travel_time_awal'] = travel_time_awal
df['travel_time'] = travel_time_awal
df['flow'] = [0,0,0]
df
# edge_attr = df_attributes_only.set_index('id').to_dict('index')

In [ ]:
# nx.set_edge_attributes(G, values = df['travel_time'].values, name="travel_time")
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr=True, create_using=nx.MultiDiGraph())
G[1][2]

In [ ]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

In [ ]:
minimum = G[1][2][0]['travel_time']
keys = 0
for index, i in enumerate(G[1][2]):
    if G[1][2][i]['travel_time'] < minimum:
        minimum = G[1][2][i]
        keys = index

keys

In [ ]:
def routes_jalan(G, routes):
    list_jalan = []
    for i in range(len(routes)-1):
        minimum = G[routes[i]][routes[i+1]][0]['travel_time']
        n = 0
        keys = 0
        for node1, node2, data in G.edges(data=True):
            if data['travel_time'] < minimum:
                minimum = data['travel_time']
                keys = n
            n += 1
        list_jalan.append(keys)
    
    return list_jalan

In [ ]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

## Updating Flow

In [ ]:
nx.set_edge_attributes(G, values = 0, name='auxflow')

In [ ]:
def updating_mainflow(G, routes):

    rute_jalan_key = routes_jalan(G, routes)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"flow": flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [ ]:
def updating_auxflow(G, routes):

    rute_jalan_key = routes_jalan(G, routes)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"auxflow": flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [ ]:
nx.to_pandas_edgelist(G)

In [ ]:
Gupdate = updating_mainflow(G, routes)
df = nx.to_pandas_edgelist(Gupdate)
df

In [ ]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr_idn(row):
    if np.isnan(row['flow']):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+0.15*(row['flow']/row['capacity'])**4)
        except ValueError:
            return row['travel_time']

In [ ]:
list_edge = []
for i in G.edges():
    list_edge.append(i)

for index, i in enumerate(list_edge):
    attrib = {(i[0], i[1], index): {"travel_time": lpr_idn(G[i[0]][i[1]][index])}}
    nx.set_edge_attributes(G, attrib)

nx.to_pandas_edgelist(G)

In [ ]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

In [ ]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

In [ ]:
Gupdate = updating_auxflow(G, routes)
nx.to_pandas_edgelist(Gupdate)

## Bisection Method

In [ ]:
def line_search(row, a, alpha, beta):
    t=row['travel_time_awal']
    x=row['flow']
    y=row['auxflow']
    c=row['capacity']
    return -t*(x-y)*(alpha*(((x+a*(y-x))/c)**beta)+1)

In [ ]:
def bisection(G, xl=0, xr=1, delta=0.0002, alpha=0.15, beta=4):
    n = 0
    df = nx.to_pandas_edgelist(G)
    condition = True
    while condition:
        n += 1
        x = (xl+xr)/2
        if df.apply(lambda row: line_search(row, x, alpha=alpha, beta=beta), axis = 1).sum() <= 0:
            xl = x
        else:
            xr = x
        condition = abs(xr-xl) > 2*delta
        print('|xl-xr| =', abs(xr-xl),' xl =', xl, ' xr=',xr)
    print(f"number of iteration in bisection method is {n}")
    return (xr+xl)/2

In [ ]:
nx.to_pandas_edgelist(G)

In [ ]:
bisection(G)